# Setting up the model

In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [2]:
from utils.model import create_effnetb3_model
effnetB3, effnetB3_transform  = create_effnetb3_model()

In [3]:
from torchvision import transforms
image_transform = transforms.Compose([transforms.Grayscale(num_output_channels=3),
                                                  effnetB3_transform])

In [4]:
from torchinfo import summary
summary(effnetB3,
        input_size=(1, 3, 320, 320),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 320, 320]     [1, 10]              --                   Partial
├─Sequential (features)                                      [1, 3, 320, 320]     [1, 1536, 10, 10]    --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 320, 320]     [1, 40, 160, 160]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 320, 320]     [1, 40, 160, 160]    (1,080)              False
│    │    └─BatchNorm2d (1)                                  [1, 40, 160, 160]    [1, 40, 160, 160]    (80)                 False
│    │    └─SiLU (2)                                         [1, 40, 160, 160]    [1, 40, 160, 160]    --                   --
│    └─Sequential (1)                                        [1, 40, 160, 160]    [1, 2

# Getting Data

In [5]:
from torchvision import datasets

In [6]:
train_data = datasets.MNIST(root = "data", train = True, transform = image_transform, download = True)
test_data = datasets.MNIST(root = "data", train = False, transform = image_transform, download = True)

In [7]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(
                              dataset = train_data,                          
                              batch_size= 32,
                              shuffle = True,
                              )
test_dataloader =  DataLoader(
                              dataset=test_data,
                              batch_size= 32,
                              shuffle = False,
                              )

# Create the model

In [14]:
from utils.model import create_effnetb3_model
effnetB3, transforms = create_effnetb3_model()

In [16]:
from torchinfo import summary

summary(effnetB3,
        input_size=(32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 10]             --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1536, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 40, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 40, 112, 112]   (1,080)              False
│    │    └─BatchNorm2d (1)                                  [32, 40, 112, 112]   [32, 40, 112, 112]   (80)                 False
│    │    └─SiLU (2)                                         [32, 40, 112, 112]   [32, 40, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 40, 112, 112]   [32, 

# Training the model

In [17]:
import torch
from utils import engine

loss_fn = torch.nn.CrossEntropyLoss(label_smoothing = 0.1)
optimizer = torch.optim.Adam(params = effnetB3.parameters(), lr=5e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,"min",patience=5)
early_stopping = engine.EarlyStopping(tolerance=5, min_delta=10)

In [18]:
from utils import engine
EPOCHS = 1

results = engine.train(model=effnetB3,
                      train_dataloader=train_dataloader,
                      test_dataloader=test_dataloader,
                      epochs=EPOCHS,
                      optimizer=optimizer,
                      loss_fn=loss_fn,
                      scheduler = scheduler,
                      device=device)

  0%|          | 0/1 [00:00<?, ?it/s]

# Saving Model

In [ ]:
from utils.util import save_model
save_model(model = mobileNetV3,
                   target_dir = "models",
                   model_name = f"mobileNetV3_quickdraw_animals_epoch_{EPOCHS}.pth")